In [18]:
p_storage_dir = "../../storage"
p_notebooks_root = None

In [19]:
if p_notebooks_root is not None:
    import os
    os.chdir(p_notebooks_root)

In [20]:
from ast import literal_eval

import pandas as pd
from pathlib import Path

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Palavras Chaves

In [21]:
palavras_chaves = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/palavras_chaves.csv", delimiter=";")

palavras_chaves.head(n=5)

,palavra_chave
0,"[{'palavra': 'goiania', 'correta': 'Goiânia', ..."
1,"[{'palavra': 'enfermagem', 'correta': 'Enferma..."
2,"[{'palavra': 'engenharia', 'correta': 'Engenha..."
3,"[{'palavra': 'administracao', 'correta': 'Admi..."
4,"[{'palavra': 'administracao', 'correta': 'Admi..."


In [22]:
def extract_palavras(row, result_list):
    palavras = literal_eval(row.values[0])
    
    for palavra in palavras:
        del palavra['importancia']
        result_list.append(palavra)

In [23]:
palavras_extracted = []

palavras_chaves.apply(extract_palavras, args=(palavras_extracted,), axis=1)

palavras_chaves_final = pd.DataFrame(palavras_extracted)

palavras_chaves_final.sample(n=10)

,palavra,correta
33944,tendencias pesquisas,Tendências Das Pesquisas
113367,cultura,Cultura
38161,aprendizados,Aprendizados
51391,estado,Estado
54287,particulas sedimentacao,Partículas Em Sedimentação
68688,pocos escavados,Poços Escavados
31121,ciencia politica,Ciência Política
43336,curriculo,Currículo
88906,centro,Centro
81127,entorno,Entorno


In [24]:
palavras_chaves_final = palavras_chaves_final.drop_duplicates(subset='palavra')

In [25]:
palavras_chaves_final.sample(n=10)

,palavra,correta
8581,manta termica,Manta Térmica
13380,historia social,História Social
68701,atraves tematicas,Através De Temáticas
46631,influencia estacao,Influência Da Estação
106585,analise percepcoes,Análise Das Percepções
39555,mapeamento fragilidade,Mapeamento Da Fragilidade
60808,contribuir,Contribuir
112898,modellus construcao,Modellus Para Construção
1575,superior,Superior
244,projeto,Projeto


Limpando palavras chaves mal formadas

In [26]:
palavras_chaves_final[palavras_chaves_final['correta'].str.contains('&\S')]

,palavra,correta
23825,conectados rede,Conectados A&#768; Rede
100133,dores definidas,Dores Na&#771;O Definidas


In [27]:
palavras_chaves_final.loc[23825]['correta'] = "Conectados À Rede"

In [28]:
palavras_chaves_final.loc[100133]['correta'] = "Dores Não Definidas"

In [29]:
palavras_chaves_final[palavras_chaves_final['correta'].str.contains('&\S')]

,palavra,correta


**Exportar as palavras chaves**

In [30]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    palavras_chaves_final.to_csv(export_path / "palavras_chaves.csv", sep=";", index=False)
except Exception as e:
    print(str(e))